# Segmenting & Clustering Neighborhoods in Toronto

## Part 1 of 3 - Retrieve & normalize data

**The data is available as a Wikipedia HTML page & the page source reveals a Table Structure (Column headings & data for each row)**

```
<tbody>
<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th>
</tr>

................

<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>
</tr>

<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>
</tr>

................

</tbody>

```

In [1]:
# import the libraries
import requests                     # Import requsts library that allows us to invoke a Web service
from bs4 import BeautifulSoup       # Utilize BeautifulSoup to parse & select data 
import pandas as pd                 # Utilize pandas DataFrame to store the retrieved table from Web scraping

# retrieve data using requests.get('url')
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')


**Utilize BeautifulSoup HTML parser to retrieve the markup document & store it in a 'soup' variable**

In [2]:
# load the data into a soup variable
soup = BeautifulSoup(data.text, 'html.parser')  # Copy the HTML document to a soup variable
tbody  = soup.find('tbody')                     # Use the embedded table body (HTML tag tbody) & retrieve the table

# Optionally print the tbody to console to visualize the HTML representation of the table
#print(tbody)

**Iterate thru the body of the table & retrieve all the rows**

In [3]:
# Create a list called table[] to store the extracted values from the HTML table
table = []

# Copy the headings & data to the table
#for tr in tbody.find_all('tr'):
#    headings = [th.text for th in tr.find_all('th')]
#    table.append(headings)
#    values   = [tr.text for td in tr.find_all('td')]
#    table.append(values)

for tr in tbody.find_all('tr'):
    values   = [tr.text for td in tr.find_all('td')]  # Retrieve column values using HTML table data tag
    #values   = [tr.text for td in tr.find_all('a')]  # Retrieve column values using HTML attribute tag
    table.append(values)

# Display the number of rows retrieved from the table
print("\nTotal number of rows retrieved from the HTML table = ", len(table))

# Optionally print the raw table to the console 
#print(table)


Total number of rows retrieved from the HTML table =  288


**Cleanup the Table; Remove Empty lists, substitute newline '\n' characters with commas**

In [4]:
# Remove empty lists, tuples, zeroes, strings
table = [x for x in table if x != []]

# Print the cleaned table to the console
print("\nTotal number of rows retained from the HTML table = ", len(table))
#print(table)

# Create 
cleanTable = []

#for lst in table:
#    for element in lst:
#        if element not in cleanTable:
#                cleanTable.append(element)

for i in range(len(table)):
#    #print(table[i][0])
    cleanTable.append([str.replace(table[i][0], '\n', ',')])

# Print the cleaned table to the console
print("\nTotal number of rows in the cleaned version table = ", len(cleanTable))
#print(cleanTable)



Total number of rows retained from the HTML table =  287

Total number of rows in the cleaned version table =  287


**Copy the cleaned table (list of lists) to a DataFrame & clean up the same**

In [18]:
# Parse the cleanTable & split the comma separated values & create a dataframe

locDataFrame = pd.DataFrame()

for listItem in cleanTable:
    locDataFrame = locDataFrame.append(pd.DataFrame([sub.split(",") for sub in listItem]))

# Rename the Columns from default index (a number) to desired column names 1. Postcode 2. Borough 3. Neighborhood
locDataFrame.rename(columns={1:'Postcode', 2:'Borough', 3:'Neighborhood'}, inplace=True)

# Drop the 1st & last columns that don't have any values
locDataFrame.drop(columns=[0,4], inplace=True)

# Sort the Dataframe by Postcode, then by Borough, & by Neighborhood
locDataFrame.sort_values(['Postcode', 'Borough', 'Neighborhood'], axis=0, inplace = True)

# Rest the index (it'll insert a column named 'index' & assign the indexed values to the 'index' )
locDataFrame.reset_index(inplace=True)

# Drop the index column of DataFrame since they are all zeroes anyway
locDataFrame.drop(columns=['index'], inplace=True)

# Drop the rows that do not have an assigned 'Borough' name & copy it to a new location DataFrame
nlDataFrame = locDataFrame[locDataFrame['Borough'] != 'Not assigned'] 

print("\nTotal number of rows filtered (after removing ones with no Borough names)  ==> ", len(nlDataFrame))

# Print all the rows that contain 'Not assigned' (also a sanity check for any Burroughs that slipped thru)
print("\n\n== Rows with unassigned Neighborhood names ==\n\n", nlDataFrame[nlDataFrame.eq('Not assigned').any(1)])



Total number of rows filtered (after removing ones with no Borough names)  ==>  210


== Rows with unassigned Neighborhood names ==

     Postcode       Borough  Neighborhood
249      M9A  Queen's Park  Not assigned


**Fix the unassigned values, and aggregate all the Neighborhood by Postcode & Borough**  

In [19]:
# Loop through the dataframe, copy the Borough name to Neighborhood name if the latter is unassigned
for idx in nlDataFrame.index:
    if(nlDataFrame['Neighborhood'].at[idx] == 'Not assigned'):
        print("\nBorough name of the unassigned neighborhood ( at index",idx,") ==> ", nlDataFrame['Borough'].at[idx])
        nlDataFrame['Neighborhood'].at[idx] = nlDataFrame['Borough'].at[idx]
        print("\nNeighborhood name reset to the Borough name ( at index",idx,") ==> ", nlDataFrame['Neighborhood'].at[idx])


# Display the head of the dataframe (notice it's automatically sorted by 'Postcode')
#df_filtered.head()
        
#############################################################################################################
# CONSOLIDATE THE DATAFRAME BY AGGREGATING NEIGHBORHOOD VALUES, FOR EACH POSTCODE, FOLLOWED BY BOROUGHNAME
#############################################################################################################

nnlDataFrame = nlDataFrame.groupby(['Postcode', 'Borough']).agg(lambda col: ','.join(col))

# Reset the Index
nnlDataFrame.reset_index(inplace = True)

# Print the head of the new consolidated dataframe (first 12 rows)
nnlDataFrame.head(12)



Borough name of the unassigned neighborhood ( at index 249 ) ==>  Queen's Park

Neighborhood name reset to the Borough name ( at index 249 ) ==>  Queen's Park


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


**Finally print the shape of the new DataFrame object (as required to show as final step)**

In [20]:
print("The Shape of the DataFrame is ==> ", nnlDataFrame.shape)

The Shape of the DataFrame is ==>  (103, 3)


## Part 2 of 3 - Enrich the Dataframe & analyze data

**Installation of geocoder package did not respond & hence the provided CSV file will be used for latitude & longitude info**
```
#!conda install -c conda-forge geocoder
#!conda install -c conda-forge/label/gcc7 geocoder
#!conda install -c conda-forge/label/cf201901 geocoder

#import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = "M5G"
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print (latitude, longitude)

```

**Retrieve the CSV file with Postal Code, Latitude & Longitude & read it into a pandas DataFrame**

In [8]:
# Read the CSV file into a pandas DataFrame
geoDataFrame = pd.read_csv('http://cocl.us/Geospatial_data')

# Display the head of the DataFrame
geoDataFrame.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Rename the 'Postal Code' to 'Postcode' so that the DataFrames can be joined on 'Postcode' as the key**

In [9]:
# Rename the key column of the DataFrame
geoDataFrame.rename(columns={'Postal Code':'Postcode'}, inplace=True)

# Display the head of the DataFrame with renamed column
geoDataFrame.head()


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Merge the DataFrames using Postcode as the key & create a final location DataFrame
flDataFrame = pd.merge(nnlDataFrame, geoDataFrame, on='Postcode')

# Display the shape of the Merged DataFrame (should be same as the shape of DataFrame constructed in part 1 of this exercise) 
print("The shape of the merged DataFrame is ==> ", flDataFrame.shape)

# Display the head (1st 12 rows) of the DatFrame 
flDataFrame.head(12)


The shape of the merged DataFrame is ==>  (103, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Part 3 of 3 - Segment / cluster, & visualize data